### Food Image Classification With a CNN

In [1]:
# load the necessary packages
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

import os
import numpy as np
import matplotlib.pyplot as plt

### Load and Preprocess Data for Model

In [2]:
batch_size = 150
epochs = 15
IMG_HEIGHT = 180
IMG_WIDTH = 180
train_dir = 'data/resized/train'
test_dir = 'data/resized/test'
dropout_rate = 0.15

In [3]:
train_image_generator = ImageDataGenerator(rescale=1./255)
test_image_generator = ImageDataGenerator(rescale=1./255)

In [4]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                          directory=train_dir,
                                                          shuffle=True,
                                                          target_size=(IMG_HEIGHT,IMG_WIDTH))

Found 75750 images belonging to 101 classes.


In [5]:
test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                          directory=test_dir,
                                                          shuffle=True,
                                                          target_size=(IMG_HEIGHT,IMG_WIDTH))

Found 25250 images belonging to 101 classes.


In [6]:
model = Sequential([
    Conv2D(64, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(rate=dropout_rate),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(rate=dropout_rate),
    Conv2D(256, 5, padding='same', activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(rate=dropout_rate),
    Conv2D(512, 5, padding='same', activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(rate=dropout_rate),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dense(101,activation=None)
])

In [7]:
model.compile(optimizer='adam',
              loss='cosine_similarity',
              metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 180, 180, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 90, 90, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 90, 90, 64)        256       
_________________________________________________________________
dropout (Dropout)            (None, 90, 90, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 90, 90, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 45, 45, 128)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 45, 45, 128)       5

In [9]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=75750//batch_size,
    epochs=epochs,
    validation_data=test_data_gen,
    validation_steps=25250//batch_size,
    workers=1,
    use_multiprocessing=False
)

Epoch 1/15
505/505 [==============================] - 753s 1s/step - loss: -0.1509 - accuracy: 0.0899 - val_loss: -0.0435 - val_accuracy: 0.0197
Epoch 2/15
505/505 [==============================] - 746s 1s/step - loss: -0.2410 - accuracy: 0.1785 - val_loss: -0.2585 - val_accuracy: 0.1937
Epoch 3/15
505/505 [==============================] - 745s 1s/step - loss: -0.3122 - accuracy: 0.2601 - val_loss: -0.3322 - val_accuracy: 0.2745
Epoch 4/15
505/505 [==============================] - 744s 1s/step - loss: -0.3865 - accuracy: 0.3441 - val_loss: -0.3582 - val_accuracy: 0.3031
Epoch 5/15
505/505 [==============================] - 742s 1s/step - loss: -0.4674 - accuracy: 0.4376 - val_loss: -0.3932 - val_accuracy: 0.3433
Epoch 6/15
505/505 [==============================] - 741s 1s/step - loss: -0.5554 - accuracy: 0.5397 - val_loss: -0.3745 - val_accuracy: 0.3223
Epoch 7/15
505/505 [==============================] - 739s 1s/step - loss: -0.6357 - accuracy: 0.6291 - val_loss: -0.4136 - val_ac

In [11]:
if not os.path.isdir('./tmp/models'):
    os.makedirs('./tmp/models')
model.save('tmp/models/CNN_MODEL_V1.h5')